# Examples of usage of Gate Angle Placeholder

The word “Placeholder” is used in Qubiter (we are in good company, Tensorflow uses this word in the same way) to mean a variable for which we delay/postpone assigning a numerical value (evaluating it) until a later time. In the case of Qubiter, it is useful to define gates with placeholders standing for angles. One can postpone evaluating those placeholders until one is ready to call the circuit simulator, and then pass the values of the placeholders as an argument to the simulator’s constructor. Placeholders of this type can be useful, for example, with quantum neural nets (QNNs). In some QNN algorithms, the circuit gate structure is fixed but the angles of the gates are varied many times, gradually, trying to lower a cost function each time.

> In Qubiter, legal variable names must be of the form `#3` or `-#3` with 3 replaced by any
    other non-negative int. The 3 is called the variable number.


In [1]:
import os
import sys
print(os.getcwd())
os.chdir('../')
print(os.getcwd())
sys.path.insert(0,os.getcwd())

/home/Notebooks/Quantum/qubiter/jupyter-notebooks
/home/Notebooks/Quantum/qubiter


We begin by writing a simple circuit with 4 qubits. As usual, the following code will
write an English and a Picture file in the `io_folder` directory. Note that some
angles have been entered into the write() Python functions as legal
variable names instead of floats. In the English file, you will see those legal
names where the numerical values of those angles would have been.

In [2]:
from SEO_writer import *
from SEO_reader import *
from EchoingSEO_reader import *
from SEO_simulator import *

In [3]:
num_bits = 4
file_prefix = 'io_folder/gate_vars_test'
emb = CktEmbedder(num_bits, num_bits)
wr = SEO_writer(file_prefix, emb)
wr.write_Rx(2, rads=np.pi/7)
wr.write_Rx(1, rads='#2')
wr.write_Rn(3, rads_list=['#1', '-#1', '#3'])
wr.write_cnot(2, 3)
wr.close_files()

The following 2 files were just written:
1. <a href='../io_folder/gate_vars_test_4_eng.txt'>../io_folder/gate_vars_test_4_eng.txt</a>
2. <a href='../io_folder/gate_vars_test_4_ZLpic.txt'>../io_folder/gate_vars_test_4_ZLpic.txt</a>

Simply by creating an object of the class SEO_reader with the flag `write_log` set equal to True, you can create a log file which contains a list of the distinct placeholder variables encountered in the English file

In [4]:
rdr = SEO_reader(file_prefix, num_bits, write_log=True)

The following log file was just written:
    
<a href='../io_folder/gate_vars_test_4_log.txt'>../io_folder/gate_vars_test_4_log.txt</a>

Partial substitution. The following code creates new English and Picture files with `#1=30, #2=60` 
but `#3` still undecided in the English file.

In [5]:
vman = PlaceholderManager(eval_all_vars=False,
            var_num_to_rads={1: np.pi/6, 2: np.pi/3})
wr = SEO_writer(file_prefix + '_eval01', emb)
EchoingSEO_reader(file_prefix, num_bits, wr,
                  vars_manager=vman)

The following 2 files were just written:
1. <a href='../io_folder/gate_vars_test_eval01_4_eng.txt'>../io_folder/gate_vars_test_eval01_4_eng.txt</a>
2. <a href='../io_folder/gate_vars_test_eval01_4_ZLpic.txt'>../io_folder/gate_vars_test_eval01_4_ZLpic.txt</a>

The following code runs the simulator after substituting
`#1=30, #2=60, #3=90`

In [6]:
vman = PlaceholderManager(
    var_num_to_rads={1: np.pi/6, 2: np.pi/3, 3: np.pi/2})
sim = SEO_simulator(file_prefix, num_bits, verbose=False,
                    vars_manager=vman)
StateVec.describe_st_vec_dict(sim.cur_st_vec_dict)

*********branch= pure
total probability of state vector (=one if no measurements)= 1.0
dictionary with key=qubit, value=(Prob(0), Prob(1))
{0: (1.0, 0.0),
 1: (0.2500000000000001, 0.7499999999999999),
 2: (0.8117449009293668, 0.1882550990706332),
 3: (0.701470365270198, 0.298529634729802)}
